In [1]:
# imports

from torchvision import transforms
from torchinfo import summary
from torch import hub, device, cuda, load, Tensor, nn, cat, from_numpy, argmax, mean
import torch
from torch.utils.data import DataLoader
from timm.data import resolve_model_data_config, create_transform

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import pickle
import numpy as np
import os
import csv
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from collections import OrderedDict
import sys
sys.path.append(r'/home/stud/afroehli/coding/util_scripts')
from utils_dataloading.imagenet_tree import ImagenetSemanticInfo, ImagenetSemanticSubtree

device = device('cuda' if cuda.is_available() else 'cpu')
print(f'Device used: {device}')

/home/stud/afroehli/miniconda3/envs/dinov2Pre/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device used: cuda


In [2]:
# util objects

imagenet_info = ImagenetSemanticInfo()

Stop node discovered


In [3]:
with open('/home/stud/afroehli/datasets/imagenet_v2_label_transform/imagenet_1k_label_order.txt', 'r') as label_order_file:
    inet_1k_labels = label_order_file.readlines()
    inet_1k_labels = [label_order_line.split()[0] for label_order_line in inet_1k_labels]

In [4]:
class_a_b_map = dict()

with open('/home/stud/afroehli/coding/dinov2_ood/storage/class_a_class_b_timm_trans.csv', 'r') as class_split_table:
    closest_pairs_timm_trans = csv.reader(class_split_table, delimiter=';')

    for class_a, class_b in closest_pairs_timm_trans:
        class_a_b_map[class_a] = class_b

In [5]:
# load model

pretr_head = load('/home/stud/afroehli/coding/dinov2_ood/pretrained_heads/dinov2_vits14_linear_head.pth')
pretr_head_big = load('/home/stud/afroehli/coding/dinov2_ood/pretrained_heads/dinov2_vits14_linear4_head.pth')

/tmp/ipykernel_1126944/2504633664.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretr_head = load('/home/stud/afroehli/coding/dinov2_ood/pretrained_heads/dinov2_vits14

In [17]:
from torch import nn

class LinearClassifier(nn.Module): 

    def __init__(self, in_features = 384, out_features = 1000):
        super().__init__()

        self.network = nn.Sequential(
            nn.Linear(in_features= in_features, out_features=out_features),
            nn.Softmax(dim=1)
        )

    # def forward(self, x):
    #     return cat((self.network.forward(x), self.network[0](x)))

    def forward(self, x):
        return self.network(x)
    

In [7]:
# DataSet for training LinearClassifier
# custom dataset: dictonary -> [(embedding, index_as_tensor)]

class DictionaryDataset(torch.utils.data.Dataset): 
    
    """
    Pararms:
    1) data: complete dataset provided as dict
    2) index_list: list of data_dict-keys, order of keys in list will be used to create a tensor as the expected model-output
    """

    def __init__(self, data: dict, index_list: list[str]): 
        self.data_dict = data
        self.index_list = index_list 
        self.wnid_list = list(self.data_dict.keys())
        self.wnid_iterator = iter(self.wnid_list)
        self.instance_per_wnid = [len(self.data_dict[key]) for key in self.wnid_list]

    def __len__(self) -> int: 
        total_len = 0 
        for key in self.wnid_list:
            total_len += len(self.data_dict[key])

        return total_len 
    
    def __getitem__(self, idx) -> tuple[torch.Tensor, torch.Tensor]: 

        sum_instances = 0
        sum_rest = 0

        # start with first wnid in list
        new_wnid = ''

        # iterate until total of all already covered instances is bigger than index of interest
        # when class of interest is reached, condition will be false (sum_instances points to first element of next class)
        for wnid in self.wnid_list: 
            if sum_instances > idx:
                break 
            new_wnid = wnid 
            sum_rest = sum_instances 
            sum_instances += len(self.data_dict[wnid])

        # index within class is needed 
        # fixed order: [class1, class2, ...]
        # when lenght of each previously covered class is known, index within class of interest can be calculated 
        # sum_rest always represent the total of instances of all covered classes yet, not of interest
        idx_within_class = idx - sum_rest

        np_arr = self.data_dict[new_wnid][idx_within_class]
        data_tensor = torch.tensor(np.array(np_arr))

        label_tensor = torch.zeros(len(self.index_list))
        # put a one at the position for the expected class 
        label_tensor[self.index_list.index(new_wnid)] = 1 

        return data_tensor, label_tensor

In [8]:
# create all needed dataloaders

dataloaders_dict = dict()

for loader_name, loader_path in [('inet_1k_val', '/home/stud/afroehli/coding/model_results/dinov2_vits14/inet_1k_val_pt_timm_trans.pkl'),
                                 ('inet_r', '/home/stud/afroehli/coding/model_results/dinov2_vits14/inet_r_plus_pt_timm_trans.pkl'),
                                 ('inet_v2_70', '/home/stud/afroehli/coding/model_results/dinov2_vits14/inet_v2_70_plus_pt_timm_trans.pkl'),
                                 ('inet_v2_mf', '/home/stud/afroehli/coding/model_results/dinov2_vits14/inet_v2_mf_plus_pt_timm_trans.pkl'),
                                 ('inet_v2_top', '/home/stud/afroehli/coding/model_results/dinov2_vits14/inet_v2_top_plus_pt_timm_trans.pkl')]:
    
    with open(loader_path, 'rb') as pkl_fl:
        dataloaders_dict[loader_name] = DataLoader(dataset=DictionaryDataset(data=pickle.load(pkl_fl), index_list=inet_1k_labels), 
                                                   batch_size=128, shuffle=False, num_workers=8, pin_memory=True)



In [9]:
dataloaders_inet_c = dict()

src_path = '/home/stud/afroehli/coding/model_results/dinov2_vits14/imagenet_c'
inet_c_cor_types = os.listdir(src_path)

for cor_type in inet_c_cor_types:
    for sev in range(1, 6):
        with open(f'{src_path}/{cor_type}/sev_{sev}.pkl', 'rb') as pkl_fl: 
            if sev == 1:
                dataloaders_inet_c[cor_type] = dict()
            dataloaders_inet_c[cor_type][sev] = DataLoader(dataset=DictionaryDataset(pickle.load(pkl_fl), inet_1k_labels), 
                                                           batch_size=128, shuffle=False, num_workers=8, pin_memory=True) 

In [23]:
# helper function 
# takes: model and list of dataloaders
# returns: accuracy for each dataloader

def cls_with_patch_one_layer(batch_samples: torch.Tensor) -> torch.Tensor:
    return cat([cat((batch_sample[3][1].squeeze(), batch_sample[3][0].squeeze())).unsqueeze(0)
                                            for batch_sample in batch_samples])

def cls_with_patch_one_layer_inet_c(batch_samples: torch.Tensor) -> torch.Tensor: 
    return cat([cat((batch_sample[1].squeeze(), batch_sample[0].squeeze())).unsqueeze(0) for batch_sample in batch_samples])

def cls_without_patch_one_layer(batch_samples: torch.Tensor) -> torch.Tensor: 
    return cat([batch_sample[3][1] for batch_sample in batch_samples])

def cls_without_patch_one_layer_inet_c(batch_samples: torch.Tensor) -> torch.Tensor: 
    return cat([batch_sample[1] for batch_sample in batch_samples])

def cls_with_patch_four_layers(batch_samples: torch.Tensor) -> torch.Tensor: 
    return cat([cat(((cat([cls_token for _, cls_token in batch_sample], dim=-1),
                    batch_sample[3][0])), dim=-1)
                    for batch_sample in batch_samples])

def calc_accuracy(model: torch.nn.Sequential, n_layers: int, with_patch: bool, dataloaders: list, is_inet_c_data: bool) -> list: 

    if n_layers == 1:
        if with_patch:
            sample_transform = cls_with_patch_one_layer 
            if is_inet_c_data:
                sample_transform = cls_with_patch_one_layer_inet_c 
        else:
            sample_transform = cls_without_patch_one_layer 
            if is_inet_c_data:
                sample_transform = cls_without_patch_one_layer_inet_c
    elif n_layers == 4: 
        sample_transform = cls_with_patch_four_layers 

    if sample_transform == None:
        raise ValueError('SampleTransform function could not be defined')
    
    accuracies = []
    
    for dataloader in (pbar := tqdm(dataloaders, ncols=100)):
                  
        total_preds = 0
        total_preds_true = 0
        for batch_samples, batch_labels in dataloader: 
            batch_samples_dev = sample_transform(batch_samples).to(device)
            
            model_pred = model(batch_samples_dev).cpu()

            is_equal = model_pred.argmax(axis=1) == batch_labels.argmax(axis=1)
            total_preds += is_equal.shape[0]
            total_preds_true += is_equal.type(torch.float).sum().item()

        accuracies.append(total_preds_true / total_preds)

    return accuracies 
            

In [18]:
# definition of linear classifiers 

model_params_one_layer = OrderedDict()
model_params_four_layers = OrderedDict()
model_params_cls_token = OrderedDict()
model_params_one_layer['network.0.weight'] = pretr_head['weight']
model_params_one_layer['network.0.bias'] = pretr_head['bias']
model_params_cls_token['network.0.weight'] = pretr_head['weight'][:,0:384]
model_params_cls_token['network.0.bias'] = pretr_head['bias']
model_params_four_layers['network.0.weight'] = pretr_head_big['weight']
model_params_four_layers['network.0.bias'] = pretr_head_big['bias']

lc_one_layer = LinearClassifier(in_features=768, out_features=1000)
lc_cls_token = LinearClassifier(in_features=384, out_features=1000)
lc_four_layer = LinearClassifier(in_features=1920, out_features=1000)

lc_one_layer.load_state_dict(model_params_one_layer)
lc_cls_token.load_state_dict(model_params_cls_token)
lc_four_layer.load_state_dict(model_params_four_layers)

lc_one_layer.eval()
lc_cls_token.eval()
lc_four_layer.eval()

lc_one_layer.to(device)
lc_cls_token.to(device)
lc_four_layer.to(device)

LinearClassifier(
  (network): Sequential(
    (0): Linear(in_features=1920, out_features=1000, bias=True)
    (1): Softmax(dim=1)
  )
)

In [24]:
# Calculate acc for each Imagenet-C Val dataset 

inet_c_accuracies = dict()
inet_c_dloader_list = []

for cor_type in dataloaders_inet_c.keys():
    for sev in dataloaders_inet_c[cor_type].keys():
        inet_c_dloader_list.append(dataloaders_inet_c[cor_type][sev])

inet_c_acc_list = calc_accuracy(lc_cls_token, 1, False, inet_c_dloader_list, is_inet_c_data=True)

acc_res_i = 0
for cor_type in dataloaders_inet_c.keys():
    for sev in dataloaders_inet_c[cor_type].keys():
        if sev == 1:
            inet_c_accuracies[cor_type] = dict()
        inet_c_accuracies[cor_type][sev] = inet_c_acc_list[acc_res_i]
        acc_res_i += 1 

with open('/home/stud/afroehli/coding/dinov2_ood/storage/lc_cls_pretr_inet_c_res.pkl', 'wb') as pkl_fl:
    pickle.dump(inet_c_accuracies, pkl_fl, pickle.HIGHEST_PROTOCOL)

100%|███████████████████████████████████████████████████████████████| 95/95 [11:50<00:00,  7.48s/it]


In [30]:
# one DF with acc of all OOD datasets 

inet_c_acc_dfs = dict()

for model, load_path in [('lc_with_patch', '/home/stud/afroehli/coding/dinov2_ood/storage/lc_one_lay_pretr_inet_c_res.pkl'),
              ('lc_cls', '/home/stud/afroehli/coding/dinov2_ood/storage/lc_cls_pretr_inet_c_res.pkl')]: 
    # load dict with acc of 95 Inet-C Datasets 
    with open(load_path, 'rb') as pkl_fl:
        inet_c_acc_results = pickle.load(pkl_fl) 

    # create tuples with form: (dataset, cor, sev, model, acc)
    inet_c_df_data = []
    for cor_type in inet_c_acc_results.keys(): 
        for sev in inet_c_acc_results[cor_type].keys(): 
            acc = inet_c_acc_results[cor_type][sev]
            inet_c_df_data.append(('inet-c', cor_type, sev, model, acc))

    # create DF 
    inet_c_acc_dfs[model] = inet_c_acc_df_with_patch = pd.DataFrame(data=inet_c_df_data, columns=['dataset', 'cor_type', 'sev', 'model', 'acc'])
    print(inet_c_acc_dfs[model].head())

  dataset cor_type  sev          model      acc
0  inet-c  spatter    1  lc_with_patch  0.78862
1  inet-c  spatter    2  lc_with_patch  0.75822
2  inet-c  spatter    3  lc_with_patch  0.71746
3  inet-c  spatter    4  lc_with_patch  0.72546
4  inet-c  spatter    5  lc_with_patch  0.67486
  dataset cor_type  sev   model      acc
0  inet-c  spatter    1  lc_cls  0.72488
1  inet-c  spatter    2  lc_cls  0.69854
2  inet-c  spatter    3  lc_cls  0.66534
3  inet-c  spatter    4  lc_cls  0.66916
4  inet-c  spatter    5  lc_cls  0.63050


In [66]:
# bring inet-c-acc-DF in form (datasetname + sev, model-1-acc, model-2-acc)


inet_c_avg_acc = inet_c_acc_dfs['lc_with_patch'].groupby(['sev'])[['acc']].mean()
inet_c_avg_acc.rename({'acc': 'patch-acc'}, axis='columns', inplace=True)
acc_cls = inet_c_acc_dfs['lc_cls'].groupby(['sev'])[['acc']].mean()
inet_c_avg_acc['cls-acc'] = acc_cls['acc']
inet_c_avg_acc = inet_c_avg_acc.assign(dataset= lambda x: x.index)
inet_c_avg_acc['dataset'] = inet_c_avg_acc['dataset'].apply(lambda x: 'inet-c-sev-' + str(x))
print(inet_c_avg_acc)

     patch-acc   cls-acc       dataset
sev                                   
1     0.739889  0.677464  inet-c-sev-1
2     0.673617  0.615355  inet-c-sev-2
3     0.599708  0.548516  inet-c-sev-3
4     0.497449  0.456148  inet-c-sev-4
5     0.367285  0.338309  inet-c-sev-5


In [57]:
# calculate results for inet-r, inet-v2-70, inet-v2-mf, inet-v2-top 

ood_acc = dict() 

for model, model_name in [(lc_one_layer, 'lc_with_patch'), (lc_cls_token, 'lc_cls')]: 
    ood_acc[model_name] = dict()
    for dname in dataloaders_dict.keys(): 
        if model_name == 'lc_with_patch':
            ood_acc[model_name][dname] = calc_accuracy(model, 1, True, [dataloaders_dict[dname]], is_inet_c_data=False)[0]
        else: 
            ood_acc[model_name][dname] = calc_accuracy(model, 1, False, [dataloaders_dict[dname]], is_inet_c_data=False)[0]


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.49s/it]


In [ ]:
# store ood-acc results 

with open('/home/stud/afroehli/coding/dinov2_ood/storage/lc_cls_pretr_ood_dsets_res.pkl', 'wb') as pkl_fl:
    pickle.dump(ood_acc, pkl_fl, pickle.HIGHEST_PROTOCOL)


In [70]:
# create DF for ood-acc-dict: (dset, model, acc)

ood_acc_data = []
for model_name in ood_acc.keys():
    for dset_name in ood_acc[model_name].keys():
        ood_acc_data.append((dset_name, model_name, ood_acc[model_name][dset_name]))

ood_acc_df = pd.DataFrame(data=ood_acc_data, columns=['dataset', 'model', 'acc'])
acc_with_patch = ood_acc_df.loc[ood_acc_df['model']=='lc_with_patch']
acc_cls = ood_acc_df.loc[ood_acc_df['model']=='lc_cls']
acc_cls = acc_cls.set_index([pd.Index([0, 1, 2, 3, 4])])
acc_ood_df = acc_with_patch[['dataset', 'acc']]
acc_ood_df.rename(mapper={'acc': 'patch-acc'}, axis='columns', inplace=True)
acc_ood_df['cls-acc'] = acc_cls['acc']

# concatenate inet-c results 
acc_ood_all = pd.concat([inet_c_avg_acc, acc_ood_df])
print(acc_ood_all.head(n=15))

with open('/home/stud/afroehli/coding/dinov2_ood/storage/ood_acc_results.csv', 'w', newline='') as csv_fl:
    acc_ood_all.to_csv(csv_fl, sep=';')

   patch-acc   cls-acc       dataset
1   0.739889  0.677464  inet-c-sev-1
2   0.673617  0.615355  inet-c-sev-2
3   0.599708  0.548516  inet-c-sev-3
4   0.497449  0.456148  inet-c-sev-4
5   0.367285  0.338309  inet-c-sev-5
0   0.817740  0.746440   inet_1k_val
1   0.406200  0.424333        inet_r
2   0.796900  0.729400    inet_v2_70
3   0.727700  0.668600    inet_v2_mf
4   0.832600  0.763500   inet_v2_top


/tmp/ipykernel_1126944/1684277008.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_ood_df.rename(mapper={'acc': 'patch-acc'}, axis='columns', inplace=True)
